<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/DeepLearning/3_Redes_Multicapa/ejercicios/ejercicios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Ejercicios Clase 3

En este notebook vamos a usar MLPs para generar un modelo clasificador sobre FashionMNIST así que muchas de las funciones que usamos en los ejercicios de la clase 2 te serán muy útiles.

## Ejercicio 1:

Generar un modelo perceptron multicapa con 2 capas ocultas de 512 y 128 neuronas respectivamente para clasificación sobre el dataset FashionMNIST

In [1]:
import torch
from torch import nn
import torchvision
#from IPython import display
from torchvision import transforms
from torch.utils import data


net = nn.Sequential(nn.Flatten(),
                    nn.Linear(784, 512),
                    nn.ReLU(),
                    nn.Linear(512, 128),
                    nn.ReLU(),
                    nn.Linear(128, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights);

## Ejercicio 2

Entrene el modelo por 10 épocas con un tamaño de lote de 256 y un learning rate de 0.3. (Le recomendamos reutilizar las funciones modularizadas de los ejercicios de la clase 2)

In [2]:
#ingresa tu código aquí
batch_size, lr, num_epochs = 256, 0.3, 10
loss = nn.CrossEntropyLoss(reduction='none')
trainer = torch.optim.SGD(net.parameters(), lr=lr)


In [3]:
def load_data_fashion_mnist(batch_size, resize=None):
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(
        root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(
        root="../data", train=False, transform=trans, download=True)
    return (data.DataLoader(mnist_train, batch_size, shuffle=True,
                            num_workers=1),
            data.DataLoader(mnist_test, batch_size, shuffle=False,
                            num_workers=1))

In [4]:
def accuracy(y_hat, y):
    """Compute the number of correct predictions."""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [5]:
train_iter, test_iter = load_data_fashion_mnist(batch_size)

In [6]:
def train_epoch(train_iter,net,loss_fn, optimizer,accuracy_fn, trackers):
    for X, y in train_iter:
        optimizer.zero_grad()
        y_hat = net(X)
        l = loss_fn(y_hat, y)
        l.mean().backward()
        optimizer.step()
        with torch.no_grad():
            trackers['epoch_loss'] += l.sum()
            trackers['epoch_acc'] += accuracy_fn(y_hat, y)
            trackers['num_samples'] += len(X)
    

def test_epoch(test_iter,net,accuracy_fn, trackers):
    for X, y in test_iter:
        with torch.no_grad():
            y_hat = net(X)
            trackers['test_acc'] += accuracy_fn(y_hat, y)
            trackers['num_test_samples'] += len(X)
            

In [7]:
def train(net, train_iter, test_iter, loss, num_epochs, trainer, accuracy):
    trackers = {'epoch_loss': 0.,
                'epoch_acc': 0.,
                'num_samples': 0.,
                'test_acc': 0.,
                'num_test_samples': 0.
                }
    
    for epoch in range(num_epochs):
        trackers['epoch_loss'] = 0.
        trackers['epoch_acc'] = 0.
        trackers['num_samples'] = 0.
        trackers['test_acc'] = 0.
        trackers['num_test_samples'] = 0.
        train_epoch(train_iter,net,loss, trainer,accuracy, trackers)
        test_epoch(test_iter,net,accuracy, trackers)
        print(f'Epoch {epoch+1}: loss {trackers["epoch_loss"]/trackers["num_samples"]:.3f}, accuracy {trackers["epoch_acc"]/trackers["num_samples"]:.3f}, test accuracy {trackers["test_acc"]/trackers["num_test_samples"]:.3f}')


In [8]:
train(net, train_iter, test_iter, loss, num_epochs, trainer, accuracy)

Epoch 1: loss 1.180, accuracy 0.544, test accuracy 0.722
Epoch 2: loss 0.568, accuracy 0.791, test accuracy 0.594
Epoch 3: loss 0.469, accuracy 0.828, test accuracy 0.817
Epoch 4: loss 0.418, accuracy 0.846, test accuracy 0.823
Epoch 5: loss 0.388, accuracy 0.857, test accuracy 0.797
Epoch 6: loss 0.365, accuracy 0.864, test accuracy 0.855
Epoch 7: loss 0.351, accuracy 0.872, test accuracy 0.804
Epoch 8: loss 0.335, accuracy 0.875, test accuracy 0.863
Epoch 9: loss 0.321, accuracy 0.880, test accuracy 0.803
Epoch 10: loss 0.308, accuracy 0.885, test accuracy 0.860


## Ejercicio 3 :

A partir del modelo anterior, analice que ocurre si en lugar de entrenar 10 épocas, entrena 20

In [9]:
#ingresa tu código aquí
num_epochs = 20
train(net, train_iter, test_iter, loss, num_epochs, trainer, accuracy)


Epoch 1: loss 0.301, accuracy 0.887, test accuracy 0.847
Epoch 2: loss 0.296, accuracy 0.889, test accuracy 0.874
Epoch 3: loss 0.286, accuracy 0.893, test accuracy 0.875
Epoch 4: loss 0.281, accuracy 0.895, test accuracy 0.858
Epoch 5: loss 0.269, accuracy 0.899, test accuracy 0.863
Epoch 6: loss 0.261, accuracy 0.902, test accuracy 0.875
Epoch 7: loss 0.256, accuracy 0.904, test accuracy 0.879
Epoch 8: loss 0.248, accuracy 0.908, test accuracy 0.872
Epoch 9: loss 0.240, accuracy 0.911, test accuracy 0.879
Epoch 10: loss 0.236, accuracy 0.910, test accuracy 0.879
Epoch 11: loss 0.230, accuracy 0.914, test accuracy 0.856
Epoch 12: loss 0.227, accuracy 0.914, test accuracy 0.867
Epoch 13: loss 0.756, accuracy 0.853, test accuracy 0.780
Epoch 14: loss 0.430, accuracy 0.846, test accuracy 0.851
Epoch 15: loss 0.321, accuracy 0.883, test accuracy 0.843
Epoch 16: loss 0.300, accuracy 0.889, test accuracy 0.868
Epoch 17: loss 0.277, accuracy 0.896, test accuracy 0.871
Epoch 18: loss 0.262, a

## Ejercicio 4

Aumente el learning rate a 1 y entrene nuevamente. ¿Cómo puede explicar lo que pasó?

In [10]:
lr = 1 
trainer = torch.optim.SGD(net.parameters(), lr=lr)
num_epochs = 20
train(net, train_iter, test_iter, loss, num_epochs, trainer, accuracy)

Epoch 1: loss nan, accuracy 0.108, test accuracy 0.100
Epoch 2: loss nan, accuracy 0.100, test accuracy 0.100
Epoch 3: loss nan, accuracy 0.100, test accuracy 0.100
Epoch 4: loss nan, accuracy 0.100, test accuracy 0.100
Epoch 5: loss nan, accuracy 0.100, test accuracy 0.100
Epoch 6: loss nan, accuracy 0.100, test accuracy 0.100
Epoch 7: loss nan, accuracy 0.100, test accuracy 0.100
Epoch 8: loss nan, accuracy 0.100, test accuracy 0.100
Epoch 9: loss nan, accuracy 0.100, test accuracy 0.100
Epoch 10: loss nan, accuracy 0.100, test accuracy 0.100
Epoch 11: loss nan, accuracy 0.100, test accuracy 0.100
Epoch 12: loss nan, accuracy 0.100, test accuracy 0.100
Epoch 13: loss nan, accuracy 0.100, test accuracy 0.100
Epoch 14: loss nan, accuracy 0.100, test accuracy 0.100
Epoch 15: loss nan, accuracy 0.100, test accuracy 0.100
Epoch 16: loss nan, accuracy 0.100, test accuracy 0.100
Epoch 17: loss nan, accuracy 0.100, test accuracy 0.100
Epoch 18: loss nan, accuracy 0.100, test accuracy 0.100
E

## Ejercicio 5:

Analize el efecto de cambiar las funciones de activación en el accurracy

In [11]:
#ingresa tu código aquí
net = nn.Sequential(nn.Flatten(),
                    nn.Linear(784, 512),
                    nn.Tanh(),
                    nn.Linear(512, 128),
                    nn.Tanh(),
                    nn.Linear(128, 10))

lr= 0.1
trainer = torch.optim.SGD(net.parameters(), lr=lr)
num_epochs = 10
train(net, train_iter, test_iter, loss, num_epochs, trainer, accuracy)

Epoch 1: loss 0.855, accuracy 0.721, test accuracy 0.776
Epoch 2: loss 0.525, accuracy 0.812, test accuracy 0.807
Epoch 3: loss 0.478, accuracy 0.829, test accuracy 0.779
Epoch 4: loss 0.452, accuracy 0.838, test accuracy 0.821
Epoch 5: loss 0.429, accuracy 0.847, test accuracy 0.841
Epoch 6: loss 0.415, accuracy 0.852, test accuracy 0.823
Epoch 7: loss 0.407, accuracy 0.853, test accuracy 0.797
Epoch 8: loss 0.391, accuracy 0.859, test accuracy 0.827
Epoch 9: loss 0.388, accuracy 0.860, test accuracy 0.823
Epoch 10: loss 0.377, accuracy 0.864, test accuracy 0.833


## Ejercicio 6:

Ahora genere un tercer modelo en donde ambas capas tengan 1024 neuronas. Analice si produjo algún cambio en los rendimientos.

In [12]:
#ingresa tu código aquí
et = nn.Sequential(nn.Flatten(),
                    nn.Linear(784, 1024),
                    nn.ReLU(),
                    nn.Linear(1024, 1024),
                    nn.ReLU(),
                    nn.Linear(1024, 10))

lr= 0.01
trainer = torch.optim.SGD(net.parameters(), lr=lr)
num_epochs = 20
train(net, train_iter, test_iter, loss, num_epochs, trainer, accuracy)

Epoch 1: loss 0.357, accuracy 0.872, test accuracy 0.857
Epoch 2: loss 0.354, accuracy 0.873, test accuracy 0.857
Epoch 3: loss 0.354, accuracy 0.873, test accuracy 0.859
Epoch 4: loss 0.353, accuracy 0.874, test accuracy 0.858
Epoch 5: loss 0.352, accuracy 0.874, test accuracy 0.859
Epoch 6: loss 0.351, accuracy 0.874, test accuracy 0.858
Epoch 7: loss 0.351, accuracy 0.874, test accuracy 0.859
Epoch 8: loss 0.350, accuracy 0.875, test accuracy 0.859
Epoch 9: loss 0.349, accuracy 0.875, test accuracy 0.860
Epoch 10: loss 0.348, accuracy 0.875, test accuracy 0.861
Epoch 11: loss 0.348, accuracy 0.875, test accuracy 0.860
Epoch 12: loss 0.347, accuracy 0.875, test accuracy 0.860
Epoch 13: loss 0.346, accuracy 0.876, test accuracy 0.860
Epoch 14: loss 0.345, accuracy 0.876, test accuracy 0.860
Epoch 15: loss 0.345, accuracy 0.876, test accuracy 0.861
Epoch 16: loss 0.344, accuracy 0.876, test accuracy 0.861
Epoch 17: loss 0.343, accuracy 0.877, test accuracy 0.861
Epoch 18: loss 0.343, a